In [2]:
# Nice Formatting within Jupyter Notebook
%matplotlib inline
#from IPython.display import display # Allows multiple displays from a single code-cell

# System functionality
import sys
sys.path.append('..')

# Import Here any Additional modules you use. To import utilities we provide, use something like:
#   from utils.plotter import plot_hinton

# Your Code goes here:
#from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split #, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#from sklearn import preprocessing
from numpy import unravel_index
from matplotlib.colors import ListedColormap
from numpy import linspace
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [ ]:
#####################
###   load data   ###
#####################

# load datasets
data_path = os.path.join(os.getcwd(), 'multi-spectral-classification', 'train.csv')
train = pd.read_csv(data_path, delimiter = ',')
data_path = os.path.join(os.getcwd(), 'multi-spectral-classification', 'test.csv')
test = pd.read_csv(data_path, delimiter = ',')

train = shuffle(train, random_state=399)

X_train = train.drop(['id','cropland'], axis=1).values
y_train = train['cropland'].values
X_test = test.drop(['id'], axis=1).values

print("Train: number of instances: {}, number of attributes: {}".format(train.shape[0], train.shape[1]))
print("Test: number of instances: {}, number of attributes: {}".format(test.shape[0], test.shape[1]))

In [ ]:
##########################
###   exploring data   ###
##########################

print(train.head(7))

train.describe()

In [ ]:
############################
###   standardize data   ###
############################

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)